This notebook is for running a first try of BOWave on the Frolich et. al data to see if we match their paper's results.
This requires 16 < x < 32 gb of RAM. Recommend running on Caviness with --mem-per-cpu=32gb flag set.

#Load ICs

In [12]:
%pip install matplotlib
%pip install scikit-learn

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/c2/da/a5622266952ab05dc3995d77689cba600e49ea9d6c51d469c077695cb719/matplotlib-3.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/aa/55/02c6d24804592b862b38a85c9b3283edc245081390a520ccd11697b6b24f/contourpy-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/2b/e8/61b8525acf26ec222518bdff127ae502bfa3408981fb5e5493f2b037d7fb/fonttools-4.42.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.0/151.0 kB 11.1 MB/s eta 0:00:00
  Obtaining dependency information for kiwisolver>=1.0.1 from https://files.pythonhosted.org/packages/6f/40/4ab

In [5]:
import pyrootutils

pyrootutils.set_root(path='/work/cniel/ajmeek/BOWaves/BOWaves', pythonpath=True)
#path = pyrootutils.find_root(search_from=__file__, indicator=".git")
#pyrootutils.set_root(path=path, pythonpath=True)

import BOWaves.utilities.dataloaders as dataloaders
import os
import numpy as np

In [6]:
#frolich_ics = {'ICs': np.array([]), 'labels': np.array([])}
frolich_ics = {'ICs': [], 'labels': []}

#for file in directory frolich data
frolich_data = os.listdir('../data/frolich')

#filter out subdirectories such as /img
frolich_data = [file for file in frolich_data if not os.path.isdir(file)]

for file in frolich_data:
    ICs, labels = dataloaders.load_and_visualize_mat_file_frolich('../data/frolich/' + file, visualize=False)
    frolich_ics['ICs'].extend(ICs)
    frolich_ics['labels'].extend(labels)


Now create codebooks since we have the ICs and their labels.

First split off 20% for testing.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(frolich_ics['ICs'], frolich_ics['labels'], test_size=0.2, random_state=42)

Now out of the training set, split into the different classes. Frolich's data has 4 classes.

In [10]:
if len(X_train) != len(y_train):
    raise ValueError('X_train and y_train are not the same length.')

# Forgot what the classes were. check on Caviness
neural = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}
blink = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}
muscle = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}
mixed = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}
lateyes = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}
heart = {'ICs': [], 'centroids': [], 'labels': [], 'shifts': [], 'distances': [], 'inertia': []}

for i in range(len(X_train)):
    if y_train[i] == 'neural':
        neural['ICs'].append(X_train[i])
    elif y_train[i] == 'blink':
        blink['ICs'].append(X_train[i])
    elif y_train[i] == 'muscle':
        muscle['ICs'].append(X_train[i])
    elif y_train[i] == 'mixed':
        mixed['ICs'].append(X_train[i])
    elif y_train[i] == 'lateyes':
        lateyes['ICs'].append(X_train[i])
    elif y_train[i] == 'heart':
        heart['ICs'].append(X_train[i])
    else:
        raise ValueError('Unknown class label: ' + y_train[i])

In [17]:
from BOWaves.sikmeans.sikmeans_core import shift_invariant_k_means
metric, init = 'cosine', 'random'
num_clusters = 16
centroid_len = 256
n_runs = 3
n_jobs = 1
rng = 42#np.random.RandomState(42)

#need to do this per class.
neural['centroids'], neural['labels'], neural['shifts'], neural['distances'], neural['inertia'], _ = shift_invariant_k_means(np.asarray(neural['ICs']), num_clusters, centroid_len, metric=metric, init=init, n_init=n_runs, rng=rng,  verbose=True)

blink['centroids'], blink['labels'], blink['shifts'], blink['distances'], blink['inertia'], _ = shift_invariant_k_means(blink['ICs'], num_clusters, centroid_len, metric=metric, init=init, n_init=n_runs, rng=rng,  verbose=True)

muscle['centroids'], muscle['labels'], muscle['shifts'], muscle['distances'], muscle['inertia'], _ = shift_invariant_k_means(muscle['ICs'], num_clusters, centroid_len, metric=metric, init=init, n_init=n_runs, rng=rng,  verbose=True)

mixed['centroids'], mixed['labels'], mixed['shifts'], mixed['distances'], mixed['inertia'], _ = shift_invariant_k_means(mixed['ICs'], num_clusters, centroid_len, metric=metric, init=init, n_init=n_runs, rng=rng,  verbose=True)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (210,) + inhomogeneous part.

Now that we have the codebooks, let's run the bowav clf code.
We want to do leave one subject out cross validation to try and classify the labels on the held out test set.

First, define BOWav to create the bag-of-words representations of the features learned in the codebooks.

In [ ]:
from BOWaves.sikmeans.sikmeans_core import _assignment_step

def bag_of_waves(codebooks):
    """
    Creates a bag-of-words representation of the input data using the codebooks.

    The codebooks are a list of dictionaries, where each dictionary contains the centroids, labels, shifts, distances, and raw ICs of a codebook. Therefore, they're the only thing we need to pass in.

    Parameters
    ----------
    codebooks

    Returns
    -------
    X: matrix of shape (n_ics, n_features)
        The bag-of-words representation of the input data.
    """

    #n_ics is the number of ICs from all classes in the codebooks list
    n_ics = sum(len(codebook['ICs']) for codebook in codebooks)

    #n_centroids is the number of centroids from all classes in the codebooks list
    n_centroids = sum(len(codebook['centroids']) for codebook in codebooks)

    x_squared_norms = None
    X = np.zeros((n_ics, n_centroids), dtype=codebooks[0]['centroids'].dtype)

    for ic in range(n_ics):
        for centroid in range(n_centroids):
            nu, _, _ = _assignment_step(codebooks[centroid]['ICs'][ic], codebooks[centroid]['centroids'], x_squared_norms, metric='cosine')

            nu, counts = np.unique(nu, return_counts=True)

            i_feature = nu + centroid * n_centroids
            X[ic, i_feature] = counts
    return X



In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

